The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l


# 7.3. 网络中的网络（NiN）
:label:`sec_nin`

LeNet、AlexNet 和 VGG 都有一个共同的设计模式：通过一系列的卷积层与池化层来提取空间结构特征；然后通过全连接层对特征的表征进行处理。
AlexNet 和 VGG 对 LeNet 的改进主要在于如何扩大和加深这两个模块。
或者，可以想象在这个过程的早期使用全连接层。
然而，如果使用稠密层了，可能会完全放弃表征的空间结构。
*网络中的网络* (*NiN*) 提供了一个非常简单的解决方案：在每个像素的通道上分别使用多层感知机 :cite:`Lin.Chen.Yan.2013`

## (**7.3.1. NiN块**)

回想一下，卷积层的输入和输出由四维张量组成，张量的每个轴分别对应样本、通道、高度和宽度。
另外，全连接层的输入和输出通常是分别对应于样本和特征的二维张量。
NiN 的想法是在每个像素位置（针对每个高度和宽度）应用一个全连接层。
如果我们将权重连接到每个空间位置，我们可以将其视为 $1\times 1$ 卷积层（如 :numref:`sec_channels` 中所述），或作为在每个像素位置上独立作用的全连接层。
从另一个角度看，即将空间维度中的每个像素视为单个样本，将通道维度视为不同特征（feature）。

:numref:`fig_nin` 说明了 VGG 和 NiN 及它们的块之间主要结构差异。
NiN 块以一个普通卷积层开始，后面是两个 $1\times 1$ 的卷积层。这两个$1\times 1$ 卷积层充当带有 ReLU 激活函数的逐像素全连接层。
第一层的卷积窗口形状通常由用户设置。
随后的卷积窗口形状固定为 $1 \times 1$。

![对比 VGG 和 NiN 及它们的块之间主要结构差异。](http://d2l.ai/_images/nin.svg)
:width:`600px`
:label:`fig_nin`


In [1]:
import paddle
import paddle.nn as nn
import numpy as np

class Nin(nn.Layer):
    def __init__(self, num_channels, num_filters, kernel_size, strides, padding):
        super(Nin, self).__init__()
        model = [
            nn.Conv2D(num_channels, num_filters, kernel_size, stride=strides, padding=padding),
            nn.ReLU(),
            nn.Conv2D(num_filters, num_filters, 1),
            nn.ReLU(),
            nn.Conv2D(num_filters, num_filters, 1),
            nn.ReLU()
        ]
        self.model = nn.Sequential(*model)
    def forward(self, X):
        return self.model(X)

## [**7.3.2. NiN模型**]

最初的 NiN 网络是在 AlexNet 后不久提出的，显然从中得到了一些启示。
NiN使用窗口形状为 $11\times 11$、$5\times 5$ 和 $3\times 3$的卷积层，输出通道数量与 AlexNet 中的相同。
每个 NiN 块后有一个最大池化层，池化窗口形状为 $3\times 3$，步幅为 2。

NiN 和 AlexNet 之间的一个显著区别是 NiN 完全取消了全连接层。
相反，NiN 使用一个 NiN块，其输出通道数等于标签类别的数量。最后放一个 *全局平均池化层*（global average pooling layer），生成一个多元逻辑向量（logits）。NiN 设计的一个优点是，它显著减少了模型所需参数的数量。然而，在实践中，这种设计有时会增加训练模型的时间。


In [3]:
class Net(nn.Layer):
    def __init__(self, num_channels, class_dim):
        super(Net, self).__init__()
        model = [
            Nin(num_channels, 96, 11, strides=4, padding=0),
            nn.MaxPool2D(kernel_size=3, stride=2),
            Nin(96, 256, 5, strides=1, padding=2),
            nn.MaxPool2D(kernel_size=3, stride=2),
            # Nin(256, 384, 3, strides=1, padding=1),
            # nn.MaxPool2D(kernel_size=3, stride=2),
            nn.Dropout(),
            # 标签类别数是10
            # Nin(384, 10, 3, strides=1, padding=1),
            Nin(256, 10, 3, strides=1, padding=1),
            paddle.fluid.dygraph.Pool2D(pool_type='max', global_pooling=True)
        ]
        self.model = nn.Sequential(*model)
    def forward(self, X):
        Y = self.model(X)
        Y = paddle.flatten(Y, start_axis=1)
        return Y

with paddle.fluid.dygraph.guard():
    net = Net(3, 10)
    X = paddle.to_tensor(np.random.uniform(-1., 1., [5, 3, 28, 28]).astype('float32'))
    Y = net(X)
    print(Y.shape)

[5, 10]


我们创建一个数据样本来[**查看每个块的输出形状**]。


In [4]:
with paddle.fluid.dygraph.guard():
    net = Net(1, 10)
    param_info = paddle.summary(net, (1, 1, 28, 28))
    print(param_info)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1       [[1, 1, 28, 28]]      [1, 96, 5, 5]         11,712     
    ReLU-1        [[1, 96, 5, 5]]       [1, 96, 5, 5]            0       
   Conv2D-2       [[1, 96, 5, 5]]       [1, 96, 5, 5]          9,312     
    ReLU-2        [[1, 96, 5, 5]]       [1, 96, 5, 5]            0       
   Conv2D-3       [[1, 96, 5, 5]]       [1, 96, 5, 5]          9,312     
    ReLU-3        [[1, 96, 5, 5]]       [1, 96, 5, 5]            0       
     Nin-1        [[1, 1, 28, 28]]      [1, 96, 5, 5]            0       
  MaxPool2D-1     [[1, 96, 5, 5]]       [1, 96, 2, 2]            0       
   Conv2D-4       [[1, 96, 2, 2]]       [1, 256, 2, 2]        614,656    
    ReLU-4        [[1, 256, 2, 2]]      [1, 256, 2, 2]           0       
   Conv2D-5       [[1, 256, 2, 2]]      [1, 256, 2, 2]        65,792     
    ReLU-5        [[1, 256, 2, 2]]  

## [**7.3.3训练模型**]

和以前一样，我们使用 Fashion-MNIST 来训练模型。训练 NiN 与训练 AlexNet、VGG时相似。


In [5]:
import paddle
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# 数据集处理
transform = T.Compose([
    T.Resize(64),
    T.Transpose(),
    T.Normalize([127.5], [127.5]),
])
train_dataset = FashionMNIST(mode='train', transform=transform)
val_dataset = FashionMNIST(mode='test', transform=transform)
# 模型定义
model = paddle.Model(Net(1, 10))
# 设置训练模型所需的optimizer, loss, metric
model.prepare(
    paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),
    paddle.nn.CrossEntropyLoss(),
    paddle.metric.Accuracy(topk=(1, 5)))
# 启动训练、评估
model.fit(train_dataset, val_dataset, epochs=2, batch_size=64, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/2
step 100/938 - loss: 1.7510 - acc_top1: 0.3219 - acc_top5: 0.6873 - 13ms/step
step 200/938 - loss: 1.2018 - acc_top1: 0.3822 - acc_top5: 0.7639 - 13ms/step
step 300/938 - loss: 0.8502 - acc_top1: 0.4805 - acc_top5: 0.8007 - 13ms/step
step 400/938 - loss: 0.8844 - acc_top1: 0.5520 - acc_top5: 0.8247 - 13ms/step
step 500/938 - loss: 0.7246 - acc_top1: 0.5986 - acc_top5: 0.8372 - 13ms/step
step 600/938 - loss: 0.5191 - acc_top1: 0.6321 - acc_top5: 0.8473 - 13ms/step
step 700/938 - loss: 0.6790 - acc_top1: 0.6568 - acc_top5: 0.8539 - 13ms/step
step 800/938 - loss: 0.6095 - acc_top1: 0.6762 - acc_top5: 0.8587 - 13ms/step
step 900/938 - loss: 0.6649 - acc_top1: 0.6910 - acc_top5: 0.8624 - 13ms/step
step 938/938 - loss: 0.6247 - acc_top1: 0.6953 - acc_top5: 0.8635 - 13ms/step
Eval begin...
step 100/157 - loss: 0.5220 - acc_top1: 0.8167 - acc_top5: 0.8983 - 11ms/step


## 7.3.4. 小结

* NiN使用由一个卷积层和多个 $1\times 1$ 卷积层组成的块。该块可以在卷积神经网络中使用，以允许更多的每像素非线性。
* NiN去除了容易造成过拟合的全连接层，将它们替换为全局平均池化层（即在所有位置上进行求和）。该池化层通道数量为所需的输出数量（例如，Fashion-MNIST的输出为10）。
* 移除全连接层可减少过拟合，同时显著减少NiN的参数。
* NiN的设计影响了许多后续卷积神经网络的设计。

## 7.3.5.练习

1. 调整NiN的超参数，以提高分类准确性。
1. 为什么NiN块中有两个 $1\times 1$ 卷积层？删除其中一个，然后观察和分析实验现象。
1. 计算NiN的资源使用情况。
    1. 参数的数量是多少？
    1. 计算量是多少？
    1. 训练期间需要多少显存？
    1. 预测期间需要多少显存？
1. 一次性直接将 $384 \times 5 \times 5$ 的表示缩减为 $10 \times 5 \times 5$ 的表示，会存在哪些问题？


[Discussions](https://discuss.d2l.ai/t/1869)
